In [1]:
import string as pystring

PUNCT_TO_REMOVE = pystring.punctuation + pystring.digits + "\n"
def clean_text(text):
    """custom function to removal: punctuations and digits"""
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    text = text.lower()
    return text

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("truyen_kieu.txt",sep="/", names=["Sentence"], encoding="utf8").dropna()
df

,Sentence
0,"1..Trăm năm trong cõi người ta,"
1,2..Chữ tài chữ mệnh khéo là ghét nhau.
2,"3..Trải qua một cuộc bể dâu,"
3,4..Những điều trông thấy mà đau đớn lòng.
4,"5.. Lạ gì bỉ sắc tư phong,"
...,...
3253,3250. Cũng đừng trách lẫn trời gần trời xa.
3254,"Thiện căn ở tại lòng ta,"
3255,Chữ tâm kia mới bằng ba chữ tài.
3256,"Lời quê chắp nhặt dông dài,"


In [4]:
df["Sentence"] = df["Sentence"].apply(lambda x: clean_text(x))
df

,Sentence
0,trăm năm trong cõi người ta
1,chữ tài chữ mệnh khéo là ghét nhau
2,trải qua một cuộc bể dâu
3,những điều trông thấy mà đau đớn lòng
4,lạ gì bỉ sắc tư phong
...,...
3253,cũng đừng trách lẫn trời gần trời xa
3254,thiện căn ở tại lòng ta
3255,chữ tâm kia mới bằng ba chữ tài
3256,lời quê chắp nhặt dông dài


In [5]:
corpus = df["Sentence"]
corpus

0                 trăm năm trong cõi người ta
1          chữ tài chữ mệnh khéo là ghét nhau
2                    trải qua một cuộc bể dâu
3       những điều trông thấy mà đau đớn lòng
4                       lạ gì bỉ sắc tư phong
                        ...                  
3253     cũng đừng trách lẫn trời gần trời xa
3254                  thiện căn ở tại lòng ta
3255          chữ tâm kia mới bằng ba chữ tài
3256               lời quê chắp nhặt dông dài
3257     mua vui cũng được một vài trống canh
Name: Sentence, Length: 3258, dtype: object

In [6]:
def get_centers_and_contexts (corpus, max_window_size=2):

    centers, contexts = [], []

    for line in corpus:
        line = line.split()

        if len(line) <= 2*max_window_size:
            continue

        for i in range(max_window_size, len(line)-max_window_size):
            idxs = list(range(i-max_window_size, i+max_window_size+1))
            centers.append(line[i])
            idxs.remove(i)
            
            contexts.append(" ".join([line [idx] for idx in idxs]))

    return centers, contexts

# This part is not in the code cell
centers, contexts = get_centers_and_contexts(corpus)
print(len(centers), len(contexts))

9778 9778


In [7]:
centers[:8], contexts[:8]

(['trong', 'cõi', 'chữ', 'mệnh', 'khéo', 'là', 'một', 'cuộc'],
 ['trăm năm cõi người',
  'năm trong người ta',
  'chữ tài mệnh khéo',
  'tài chữ khéo là',
  'chữ mệnh là ghét',
  'mệnh khéo ghét nhau',
  'trải qua cuộc bể',
  'qua một bể dâu'])

In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [10]:
max_length = 4
embedding_size = 200

tokenizer = Tokenizer(oov_token='<00V>')
tokenizer.fit_on_texts(corpus)

vocab_size = len(tokenizer.index_word) + 1

train_seq = tokenizer.texts_to_sequences(contexts)
train_seq_pad = pad_sequences(train_seq, maxlen=max_length, truncating='post', padding="post")

train_labels = [to_categorical(tokenizer.word_index[label], len(tokenizer.word_index) + 1) for label in centers]

train_labels = np.array(train_labels)

In [11]:
train_seq

[[143, 77, 498, 4],
 [77, 27, 4, 66],
 [126, 127, 375, 352],
 [127, 126, 352, 10],
 [126, 375, 10, 1173],
 [375, 352, 1173, 83],
 [1174, 194, 674, 185],
 [194, 2, 185, 604],
 [54, 95, 50, 22],
 [95, 62, 22, 204],
 [62, 50, 204, 864],
 [50, 22, 864, 6],
 [168, 81, 274, 464],
 [81, 1399, 464, 119],
 [33, 100, 605, 499],
 [100, 353, 499, 64],
 [353, 605, 64, 327],
 [605, 499, 327, 541],
 [1400, 757, 376, 51],
 [757, 112, 51, 311],
 [119, 16, 758, 18],
 [16, 11, 18, 500],
 [11, 758, 500, 1731],
 [758, 18, 1731, 100],
 [12, 77, 1401, 606],
 [77, 132, 606, 328],
 [205, 404, 607, 56],
 [404, 1732, 56, 212],
 [1732, 607, 212, 759],
 [607, 56, 759, 41],
 [11, 30, 608, 354],
 [30, 542, 354, 219],
 [132, 464, 9, 246],
 [464, 79, 246, 246],
 [79, 9, 246, 1733],
 [9, 246, 1733, 501],
 [2, 675, 1175, 1734],
 [675, 67, 1734, 6],
 [219, 144, 126, 432],
 [144, 10, 432, 433],
 [10, 126, 433, 1402],
 [126, 432, 1402, 132],
 [94, 6, 676, 1403],
 [6, 56, 1403, 865],
 [465, 156, 377, 275],
 [156, 10, 275, 1

In [12]:
train_seq_pad

array([[143,  77, 498,   4],
       [ 77,  27,   4,  66],
       [126, 127, 375, 352],
       ...,
       [241,   9,   2, 332],
       [  9,  89, 332, 756],
       [ 89,   2, 756, 304]])

In [13]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

In [15]:
cbow = Sequential()
cbow.add(Embedding(vocab_size, 200, input_length=train_seq_pad.shape[1]))  
cbow.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))  
cbow.add(Dense(vocab_size, activation='softmax'))  

In [16]:
cbow.compile(loss=categorical_crossentropy, optimizer=Adam())
cbow.fit(train_seq_pad, train_labels, epochs=30, verbose=1)

Epoch 1/30
306/306 [==============================] - 3s 9ms/step - loss: 7.4487
Epoch 2/30
306/306 [==============================] - 3s 9ms/step - loss: 6.7250
Epoch 3/30
306/306 [==============================] - 3s 9ms/step - loss: 6.5682
Epoch 4/30
306/306 [==============================] - 3s 9ms/step - loss: 6.4009
Epoch 5/30
306/306 [==============================] - 3s 9ms/step - loss: 6.1615
Epoch 6/30
306/306 [==============================] - 3s 9ms/step - loss: 5.8401
Epoch 7/30
306/306 [==============================] - 3s 9ms/step - loss: 5.4417
Epoch 8/30
306/306 [==============================] - 3s 9ms/step - loss: 4.9851
Epoch 9/30
306/306 [==============================] - 3s 9ms/step - loss: 4.4964
Epoch 10/30
306/306 [==============================] - 3s 9ms/step - loss: 4.0021
Epoch 11/30
306/306 [==============================] - 3s 9ms/step - loss: 3.5243
Epoch 12/30
306/306 [==============================] - 3s 9ms/step - loss: 3.0797
Epoch 13/30
306/306 [====

In [17]:
sample_text = 'trăm năm cõi người'   
sample_seq= tokenizer.texts_to_sequences ([sample_text])

sample_seq_pad = pad_sequences (sample_seq, maxlen=max_length, truncating='post', padding="post")
cbow.predict(sample_seq_pad)

1/1 [==============================] - 0s 58ms/step


array([[1.6889077e-11, 1.6512638e-11, 1.9677230e-03, ..., 7.0045857e-08,
        1.6225655e-11, 4.0489300e-07]], dtype=float32)

In [18]:
tokenizer.index_word[np.argmax(cbow.predict(sample_seq_pad))]

1/1 [==============================] - 0s 22ms/step


'trong'

In [20]:
sample_text2 = 'lời nói kịp thưa'   
sample_seq2= tokenizer.texts_to_sequences ([sample_text2])

sample_seq_pad2 = pad_sequences (sample_seq2, maxlen=max_length, truncating='post', padding="post")

tokenizer.index_word[np.argmax(cbow.predict(sample_seq_pad2))]

1/1 [==============================] - 0s 14ms/step


'chửa'